In [2]:
import cv2
import numpy as np
import math
from matplotlib import pyplot as plt

img = cv2.imread('C:\\Users\\Mauro\Desktop\\lena.jpg')
#cv2.imshow('image',img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

#print(type(img[0][0][0]))

In [4]:
b,g,r = cv2.split(img)

def getRedPattern(r):
    bp_r = np.zeros_like(r)
    for i in range(len(r)):
        if (i+1)%2 is 1:
            for j in range(len(r[i])):
                if((j+1)%2 is 1): bp_r[i][j] = r[i][j]
    return bp_r

def getGreenPattern(g):
    bp_g = np.zeros_like(g)
    for i in range(len(g)):
        if (i+1)%2 is 1:
            for j in range(len(g[i])):
                if (j+1)%2 is 0 : bp_g[i][j] = g[i][j]
        if (i+1)%2 is 0:
            for j in range(len(g[i])):
                if (j+1)%2 is 1 : bp_g[i][j] = g[i][j]
    return bp_g

def getBluePattern(b):
    bp_b = np.zeros_like(b)
    for i in range(len(b)):
        if (i+1)%2 is 0:
            for j in range(len(b[i])):
                if((j+1)%2 is 0): bp_b[i][j] = b[i][j]
    return bp_b

def PSNR(img1,img2):
    origImg = img1.astype(np.float);
    distImg = img2.astype(np.float);

    MSE = 0.0
    for i in range(len(img)-3)[3:]:
        for j in range(len(img)-3)[3:]:
            MSE = MSE + np.linalg.norm(origImg[i][j]-distImg[i][j])       
    MSE = MSE/((len(img)-6)*(len(img)-6))
    PSNR = 20*np.log10(255/np.sqrt(MSE))
    return PSNR

%time bp_r = getRedPattern(r).astype(np.float)
%time bp_g = getGreenPattern(g).astype(np.float) 
%time bp_b = getBluePattern(b).astype(np.float)

Wall time: 15.6 ms
Wall time: 15.6 ms
Wall time: 0 ns


In [5]:
def calc_mean_A(matrix,shift_r,shift_c):
    print(type(matrix))
    irange = range(shift_r,len(matrix)-1,2)
    jrange = range(shift_c,len(matrix)-1,2)
    for i in irange:
        for j in jrange:
             matrix[i][j] = (matrix[i+1][j+1]+ matrix[i+1][j-1]+ matrix[i-1][j+1]+ matrix[i-1][j-1])/4
    return matrix
                                                                        
def calc_mean_B(matrix,shift_rp,shift_rd,shift_c_rp,shift_c_rd):
    
    irange = range(shift_rp,len(matrix)-1,2)
    jrange = range(shift_c_rp,len(matrix)-1,2)
    
    for i in irange:
        for j in jrange:
            matrix[i][j] = (matrix[i+1][j]+ matrix[i-1][j]+ matrix[i][j+1]+ matrix[i][j-1])/4
            
    irange = range(shift_rd,len(matrix)-1,2)
    jrange = range(shift_c_rd,len(matrix)-1,2)
    
    for i in irange:
        for j in jrange:
            matrix[i][j] = (matrix[i+1][j]+ matrix[i-1][j]+ matrix[i][j+1]+ matrix[i][j-1])/4
            
    return matrix


In [6]:
%time bp_r = calc_mean_A(bp_r,1,1)

<class 'numpy.ndarray'>
Wall time: 32 ms


In [7]:
bp_b = calc_mean_A(bp_b,2,2)

<class 'numpy.ndarray'>


In [8]:
bp_b = calc_mean_B(bp_b,2,3,3,2)

In [9]:
bp_g = calc_mean_B(bp_g,1,2,1,2)

In [ ]:
Ricostruzione = np.zeros((len(img),len(img),3))
#Ricostruzione = np.zeros_like(img)
#print(Ricostruzione.shape)            #512x512x3
#print(type(Ricostruzione[0][0][0]))   #float64 poi devo riportarla a uint8

irange = range(len(img)-1)[1:] #1px di cornice nera poi penso al problema IndexOutOfRange

for i in irange:
    for j in irange:
        Ricostruzione[i][j][0] = bp_b[i][j] 
        Ricostruzione[i][j][1] = bp_g[i][j]
        Ricostruzione[i][j][2] = bp_r[i][j]

                
#_max = np.max(Ricostruzione)
#_min = np.min(Ricostruzione)
#print("Min;Max: ",_min,",",_max,";")
#Ricostruzione = (Ricostruzione-_min)*255/_max
#Tentavo di fare qualcosa ma il massimo è 255 ed il minimo è 0 quindi nada
      
Ricostruzione = Ricostruzione.astype(np.uint8)
cv2.imshow('image-o',img)
cv2.imshow('image-r',Ricostruzione)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
b1,g1,r1 = cv2.split(img)
b2,g2,r2 = cv2.split(Ricostruzione)
print("R:",PSNR(r1,r2),"dB")
print("G:",PSNR(g1,g2),"dB")
print("B:",PSNR(b1,b2),"dB")
print(PSNR(img,Ricostruzione),"dB")

In [27]:
Seconda = np.zeros((len(img),len(img),3))
irange = range(len(img)-3)[3:] #1px di cornice nera poi penso al problema IndexOutOfRange
#alpha = 1.0/2
#beta = 5.0/8
#gamma = 3.0/4

alpha = 1
beta = 1
gamma = 1


for i in irange:
    for j in irange:
        if (i+1)%2 is 0:
            if (j+1)%2 is 0:
                #print("elm(",i,"x",j,") is BLUE")
                Seconda[i][j][0] = bp_b[i][j]
                
                DeltaBg = (1.0/8)*(4*bp_b[i][j] - (bp_b[i-2][j]+bp_b[i+2][j]+bp_b[i][j-2]+bp_b[i][j+2]))
                DeltaBr = (1.0/8)*(6*bp_b[i][j] - (3.0/2)*(bp_b[i-2][j]+bp_b[i+2][j]+bp_b[i][j-2]+bp_b[i][j+2]))
                
                Seconda[i][j][1] = bp_g[i][j] + alpha*DeltaBg
                Seconda[i][j][2] = bp_r[i][j] + gamma*DeltaBr
                
            if (j+1)%2 is 1: 
                #print("elm(",i,"x",j,") is GREEN")
                Seconda[i][j][1] = bp_g[i][j]
                
                DeltaGr = 5*bp_g[i][j]
                DeltaGr -= (bp_g[i+1][j+1]+bp_g[i-1][j-1]+bp_g[i-1][j+1]+bp_g[i+1][j-1])
                DeltaGr -= (bp_g[i+2][j]+bp_g[i-2][j]-bp_g[i][j+2]/2-bp_g[i][j-2]/2)
                DeltaGr = DeltaGr*(1.0/8)
                
                DeltaGb = 5*bp_g[i][j]
                DeltaGb-= (bp_g[i+1][j+1]+bp_g[i-1][j-1]+bp_g[i-1][j+1]+bp_g[i+1][j-1])
                DeltaGb -= (-bp_g[i+2][j]/2-bp_g[i-2][j]/2-bp_g[i][j+2]-bp_g[i][j-2])
                DeltaGb = DeltaGr*(1.0/8)
                
                Seconda[i][j][2] = bp_r[i][j] + beta*DeltaGr
                Seconda[i][j][0] = bp_b[i][j] + beta*DeltaGb
                
        if (i+1)%2 is 1:
            if (j+1)%2 is 0: 
                #print("elm(",i,"x",j,") is GREEN")
                Seconda[i][j][1] = bp_g[i][j]
                
                DeltaGb = 5*bp_g[i][j]
                DeltaGb -= (bp_g[i+1][j+1]+bp_g[i-1][j-1]+bp_g[i-1][j+1]+bp_g[i+1][j-1])
                DeltaGb -= (bp_g[i+2][j]+bp_g[i-2][j]-bp_g[i][j+2]/2-bp_g[i][j-2]/2)
                DeltaGb = DeltaGr*(1.0/8)
                
                DeltaGr = 5*bp_g[i][j]
                DeltaGr -= (bp_g[i+1][j+1]+bp_g[i-1][j-1]+bp_g[i-1][j+1]+bp_g[i+1][j-1])
                DeltaGr -= (-bp_g[i+2][j]/2-bp_g[i-2][j]/2-bp_g[i][j+2]-bp_g[i][j-2])
                DeltaGr = DeltaGr*(1.0/8)
                
                Seconda[i][j][2] = bp_r[i][j] + beta*DeltaGr
                Seconda[i][j][0] = bp_b[i][j] + beta*DeltaGb
                
            if (j+1)%2 is 1: 
                #print("elm(",i,"x",j,") is RED")
                Seconda[i][j][2] = bp_r[i][j]
                
                DeltaRg = (1.0/8)*(4*bp_r[i][j] - (bp_r[i-2][j]+bp_r[i+2][j]+bp_r[i][j-2]+bp_r[i][j+2]))
                DeltaRb = (1.0/8)*(6*bp_r[i][j] - (3.0/2)*(bp_r[i-2][j]+bp_r[i+2][j]+bp_r[i][j-2]+bp_r[i][j+2]))
                
                Seconda[i][j][1] = bp_g[i][j] + alpha*DeltaRg
                Seconda[i][j][0] = bp_b[i][j] + gamma*DeltaRb 


Seconda = Seconda.astype(np.uint8)

cv2.imshow('image-o',img)
cv2.imshow('image-r',Seconda)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
b1,g1,r1 = cv2.split(img)
b2,g2,r2 = cv2.split(Seconda)
print("R:",PSNR(r1,r2),"dB")
print("G:",PSNR(g1,g2),"dB")
print("B:",PSNR(b1,b2),"dB")
print(PSNR(img,Seconda),"dB")

('R:', 35.096373325400208, 'dB')
('G:', 47.43549925559401, 'dB')
('B:', 43.079283682863334, 'dB')
(34.626235035257004, 'dB')
